In [ ]:
# Install the pacakge if it hasn't been installed
%pip install pot

# Code to produce Figure 5

In [1]:
import numpy as np
import ot
from scipy.stats import rdist
from sklearn.metrics import pairwise_distances
from numpy.ma.core import sqrt
from scipy.stats import chi2
from math import pi
import multiprocess as mp
NUM_CORE = 7
iter = 10000

# X, H: ndarray n by p
# Symmetry: Central, Sign, or Spherical
# Return the Wilcoxon_signed_rank statistic
def Wilcoxon_signed_rank(X, H, Symmetry="Central"):
    n, p = X.shape

    # define the cost function
    if Symmetry == "Central":
        def c_metric(x, h):
            return (-abs(np.inner(x, h)))

        def S(x, h):
            return (np.sign(np.inner(x, h)))

    if Symmetry == "Sign":
        def c_metric(x, h):
            return (-np.inner(abs(x), abs(h)))

        def S(x, h):
            return (np.sign(x * h))


    if Symmetry == "Spherical":
        x_norm = [np.linalg.norm(X[i,]) for i in range(n)]
        seq = sorted(x_norm)
        x_rank = [seq.index(v) for v in x_norm]
        h_norm = [np.linalg.norm(H[i,]) for i in range(n)]
        sorted_h = sorted(h_norm)
        W = sum([X[i,] * (sorted_h[x_rank[i]]/x_norm[i]) for i in range(n)])/sqrt(n)
        return (W)

    # loss matrix
    M = pairwise_distances(X, H, metric=c_metric, n_jobs=None)

    # Compute EMD
    G0 = ot.emd(np.ones((n,)), np.ones((n,)), M)

    W = sum([S(X[i,], H[next((i for i, x in enumerate(G0[i,]) if x > 0), None),]) * H[
        next((i for i, x in enumerate(G0[i,]) if x > 0), None),] for i in range(n)]) / sqrt(n)

    return (W)


pool = mp.Pool(NUM_CORE)

# Data distribution: Gaussian 
# 6 columns in the results correspond to CentralUnif SignUnif T2 CentralGaussian SignGaussian SphericalGaussian
Result = np.zeros((21, 6))
for j in range(21):
    n = 25 * j + 100

    def Reject(i_n):
        i, n = i_n
        np.random.seed(i)
        X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + 0.1
        # central symmetry
        H = np.random.uniform(low=0.0, high=1.0, size=(n, 2))
        H[:, 1] = H[:, 1] * 2 - 1
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        if chi2.cdf(np.inner(W, W) * 3, df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 0] = sum(pool.map(Reject, [(i, n) for i in range(iter)])) / iter


    def Reject(i_n):
        i, n = i_n
        np.random.seed(i)
        X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + 0.1
        # sign symmetry
        H = np.random.uniform(low=0.0, high=1.0, size=(n, 2))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W) * 3, df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 1] = sum(pool.map(Reject, [(i, n) for i in range(iter)])) / iter

    n0 = round(n * 3 / pi)


    def Reject(i_n):
        i, n0 = i_n
        np.random.seed(i)
        X = np.random.normal(loc=0.0, scale=1.0, size=(n0, 2)) + 0.1
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n0 * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 2] = sum(pool.map(Reject, [(i, n0) for i in range(iter)])) / iter


    def Reject(i_n):
        i, n0 = i_n
        np.random.seed(i)
        X = np.random.normal(loc=0.0, scale=1.0, size=(n0, 2)) + 0.1
        # central symmetry
        H = np.random.normal(loc=0.0, scale=1.0, size=(n0, 2))
        H[:, 0] = abs(H[:, 0])
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 3] = sum(pool.map(Reject, [(i, n0) for i in range(iter)])) / iter


    def Reject(i_n):
        i, n0 = i_n
        np.random.seed(i)
        X = np.random.normal(loc=0.0, scale=1.0, size=(n0, 2)) + 0.1
        # sign symmetry
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n0, 2)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 4] = sum(pool.map(Reject, [(i, n0) for i in range(iter)])) / iter


    def Reject(i_n):
        i, n0 = i_n
        np.random.seed(i)
        X = np.random.normal(loc=0.0, scale=1.0, size=(n0, 2)) + 0.1
        # spherical symmetry
        H = np.zeros((n0, 2))
        H[:, 0] = [sqrt(x) for x in np.random.chisquare(df=2, size=n0)]
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 5] = sum(pool.map(Reject, [(i, n0) for i in range(iter)])) / iter

    print(Result[j,])
    np.savetxt('/Users/huangzhen/desktop/HPC/Python/Gaussian.csv', Result, delimiter=',')

[0.2108 0.2172 0.2328 0.2034 0.2081 0.2164]
[0.2544 0.2594 0.2747 0.246  0.2528 0.2594]
[0.2997 0.3054 0.3218 0.2954 0.303  0.3072]
[0.346  0.3543 0.3681 0.3348 0.3466 0.3548]
[0.3898 0.3953 0.4127 0.3855 0.3923 0.3996]
[0.4296 0.4355 0.4495 0.4257 0.4379 0.4418]
[0.4688 0.4745 0.4903 0.464  0.4739 0.4778]
[0.5072 0.5163 0.529  0.5053 0.5114 0.519 ]
[0.5503 0.5589 0.5677 0.5414 0.5517 0.5579]
[0.5873 0.5946 0.6057 0.5806 0.5908 0.5943]
[0.6251 0.6291 0.6347 0.6184 0.6247 0.628 ]
[0.657  0.66   0.6701 0.6536 0.6584 0.6623]
[0.6836 0.6896 0.6939 0.679  0.6847 0.6888]
[0.7122 0.7147 0.724  0.7101 0.7133 0.7202]
[0.7406 0.7427 0.7489 0.7356 0.7404 0.7451]
[0.7638 0.7709 0.7722 0.761  0.7669 0.7693]
[0.7903 0.7943 0.7957 0.7829 0.7888 0.7928]
[0.8136 0.8162 0.8164 0.8057 0.8101 0.815 ]
[0.8325 0.8343 0.8409 0.8282 0.8315 0.8337]
[0.8497 0.8519 0.8567 0.8452 0.8498 0.8525]
[0.8611 0.8652 0.8707 0.8609 0.8664 0.8676]


In [2]:
# Data distribution: Epanechnikov 
# 6 columns in the results correspond to CentralUnif(100%) SignUnif(100%) T2(0.864) CentralGaussian(0.864) SignGaussian(0.864)
Result = np.zeros((21, 5))
for j in range(21):
    n = 25 * j + 100

    def Reject(i_n):
        i, n = i_n
        np.random.seed(i)
        X = rdist.rvs(c=4, size=(n, 2)) + 0.05
        # central symmetry
        H = np.random.uniform(low=0.0, high=1.0, size=(n, 2))
        H[:,1] = H[:,1]*2 - 1
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        if chi2.cdf(np.inner(W, W) * 3, df=2) > 0.95:
            return (1)
        else:
            return (0)

    Result[j, 0] = sum(pool.map(Reject, [(i, n) for i in range(iter)])) / iter

    def Reject(i_n):
        i, n = i_n
        np.random.seed(i)
        X = rdist.rvs(c=4, size=(n, 2)) + 0.05
        # sign symmetry
        H = np.random.uniform(low=0.0, high=1.0, size=(n, 2))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W) * 3, df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 1] = sum(pool.map(Reject, [(i, n) for i in range(iter)])) / iter

    n0 = round(n * 0.864)


    def Reject(i_n):
        i, n0 = i_n
        np.random.seed(i)
        X = rdist.rvs(c=4, size=(n0, 2)) + 0.05
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n0 * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
            return (1)
        else:
            return (0)

    Result[j, 2] = sum(pool.map(Reject, [(i, n0) for i in range(iter)])) / iter

    def Reject(i_n):
        i, n0 = i_n
        np.random.seed(i)
        X = rdist.rvs(c=4, size=(n0, 2)) + 0.05
        # central symmetry
        H = np.random.normal(loc=0.0, scale=1.0, size=(n0, 2))
        H[:, 0] = abs(H[:, 0])
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)

    Result[j, 3] = sum(pool.map(Reject, [(i, n0) for i in range(iter)])) / iter

    def Reject(i_n):
        i, n0 = i_n
        np.random.seed(i)
        X = rdist.rvs(c=4, size=(n0, 2)) + 0.05
        # sign symmetry
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n0, 2)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)


    Result[j, 4] = sum(pool.map(Reject, [(i, n0) for i in range(iter)])) / iter

    print(Result[j, ])
    np.savetxt('/Users/huangzhen/desktop/HPC/Python/Epanechnikov.csv', Result, delimiter=',')

[0.2259 0.2297 0.2509 0.2313 0.2411]
[0.2759 0.2824 0.3053 0.2912 0.3043]
[0.3222 0.332  0.3541 0.348  0.3633]
[0.3766 0.3769 0.3934 0.4034 0.4183]
[0.423  0.4337 0.4453 0.4563 0.4738]
[0.4762 0.4818 0.4937 0.5136 0.53  ]
[0.5211 0.5289 0.5389 0.5646 0.5774]
[0.5605 0.5699 0.5817 0.608  0.6216]
[0.6008 0.6073 0.6173 0.6493 0.6674]
[0.6402 0.6419 0.6544 0.6909 0.7057]
[0.6715 0.678  0.6855 0.7234 0.7359]
[0.7095 0.7157 0.7168 0.7611 0.7722]
[0.74   0.7467 0.7516 0.7945 0.8044]
[0.7695 0.7745 0.7779 0.819  0.8296]
[0.7935 0.7984 0.8017 0.8432 0.8519]
[0.8157 0.8212 0.8243 0.867  0.8738]
[0.8375 0.8412 0.8446 0.8814 0.8914]
[0.8582 0.8608 0.8632 0.8977 0.905 ]
[0.8737 0.8775 0.8785 0.9146 0.9187]
[0.8911 0.8944 0.8935 0.9258 0.9305]
[0.9028 0.9057 0.905  0.9354 0.9394]


In [ ]:
# Use SymOT.R to generate Figure 5 from the above results

## Code to implement OT-Wilcox, OT-sign for simulated examples in Appendix D.2

In [ ]:
# This block contains functions to perform OT-related tests proposed in the paper
import numpy as np
import ot
from sklearn.metrics import pairwise_distances
from math import pi
import multiprocess as mp
from scipy.stats import chi2
from scipy.stats import f
from scipy.stats import ortho_group
from scipy.stats import binom
from scipy.stats import halfnorm
from scipy.stats import norm
from scipy.stats import qmc
from datetime import datetime
NUM_CORE = 7
iter = 1000

# X, H: data points and reference vectors (ndarray n by p)
# Symmetry: Central, Sign, or Spherical
# Return the Wilcoxon_signed_rank statistic
def Wilcoxon_signed_rank(X, H, Symmetry="Central"):
    n, p = X.shape

    # define the cost function
    if Symmetry == "Central":
        def c_metric(x, h):
            return (-abs(np.inner(x, h)))

        def S(x, h):
            return (np.sign(np.inner(x, h)))

    if Symmetry == "Sign":
        def c_metric(x, h):
            return (-np.inner(abs(x), abs(h)))

        def S(x, h):
            return (np.sign(x * h))


    if Symmetry == "Spherical":
        x_norm = [np.linalg.norm(X[i,]) for i in range(n)]
        seq = sorted(x_norm)
        x_rank = [seq.index(v) for v in x_norm]
        h_norm = [np.linalg.norm(H[i,]) for i in range(n)]
        sorted_h = sorted(h_norm)
        W = sum([X[i,] * (sorted_h[x_rank[i]]/x_norm[i]) for i in range(n)])/np.sqrt(n)
        return (W)

    # loss matrix
    M = pairwise_distances(X, H, metric=c_metric, n_jobs=None)

    # Compute EMD
    G0 = ot.emd(np.ones((n,)), np.ones((n,)), M)

    W = sum([S(X[i,], H[next((i for i, x in enumerate(G0[i,]) if x > 0), None),]) * H[
        next((i for i, x in enumerate(G0[i,]) if x > 0), None),] for i in range(n)]) / np.sqrt(n)

    return (W)


def gram_schmidt(A):
    (n, m) = A.shape
    
    for i in range(m):
        
        q = A[:, i] # i-th column of A
        
        for j in range(i):
            q = q - np.dot(A[:, j], A[:, i]) * A[:, j]
        
        if np.array_equal(q, np.zeros(q.shape)):
            raise np.linalg.LinAlgError("The column vectors are not linearly independent")
        
        # normalize q
        q = q / np.sqrt(np.dot(q, q))
        
        # write the vector back in the matrix
        A[:, i] = q
    
    return (A)    
        
# X, H: ndarray n by p
# Symmetry: Central, Sign, or Spherical
# Return the p-value
def Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central"):
    n, p = X.shape

    # define the cost function
    if Symmetry == "Central":
        def c_metric(x, h):
            return (-abs(np.inner(x, h)))

        def S(x, h):
            return (np.sign(np.inner(x, h))*np.identity(p))

    if Symmetry == "Sign":
        def c_metric(x, h):
            return (-np.inner(abs(x), abs(h)))

        def S(x, h):
            return (np.diag(np.sign(x * h)))


    if Symmetry == "Spherical":
        x_norm = [np.linalg.norm(X[i,]) for i in range(n)]
        X = X[np.argsort(x_norm)]
        h_norm = [np.linalg.norm(H[i,]) for i in range(n)]
        H = H[np.argsort(h_norm)]
        Tn = 0
        for i in range(n):
            M1 = np.random.normal(loc=0.0, scale=1.0, size=(p, p))
            M1[0,:] = H[i,]
            M1 = gram_schmidt(M1)
            M2 = np.random.normal(loc=0.0, scale=1.0, size=(p, p))
            M2[0,:] = X[i,]
            M2 = gram_schmidt(M2).transpose()
            Tn = Tn + np.matmul(M1,M2)
        
        return (1 - chi2.cdf(np.sum(Tn*Tn)*p/n, df=p*p))
        

    # loss matrix
    M = pairwise_distances(X, H, metric=c_metric, n_jobs=None)

    # Compute EMD
    G0 = ot.emd(np.ones((n,)), np.ones((n,)), M)

    Tn = sum([S(X[i,], H[next((i for i, x in enumerate(G0[i,]) if x > 0), None),]) for i in range(n)])
    
    if Symmetry == "Central":
        return (1 - chi2.cdf(np.sum(Tn*Tn)/p/n, df=1))
    if Symmetry == "Sign":
        return (1 - chi2.cdf(np.sum(Tn*Tn)/n, df=p))
    
    return (W)

# X, H: ndarray n by p
# Symmetry: Central, Sign, or Spherical
# Return the p-value
def Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Central"):
    n, p = X.shape

    # define the cost function
    if Symmetry == "Central":
        def c_metric(x, h):
            return (-abs(np.inner(x, h)))

        def S(x, h):
            return (np.sign(np.inner(x, h)))

    if Symmetry == "Sign":
        def c_metric(x, h):
            return (-np.inner(abs(x), abs(h)))

        def S(x, h):
            return (np.diag(np.sign(x * h)))


    if Symmetry == "Spherical":
        x_norm = [np.linalg.norm(X[i,]) for i in range(n)]
        X = X[np.argsort(x_norm)]
        h_norm = [np.linalg.norm(H[i,]) for i in range(n)]
        H = H[np.argsort(h_norm)]
        Tn = 0
        for i in range(n):
            M1 = np.random.normal(loc=0.0, scale=1.0, size=(p, p))
            M1[0,:] = H[i,]
            M1 = gram_schmidt(M1)
            M2 = np.random.normal(loc=0.0, scale=1.0, size=(p, p))
            M2[0,:] = X[i,]
            M2 = gram_schmidt(M2).transpose()
            Tn = Tn + np.matmul(M1,M2)
        
        return (np.sum(Tn*Tn))
        

    # loss matrix
    M = pairwise_distances(X, H, metric=c_metric, n_jobs=None)

    # Compute EMD
    G0 = ot.emd(np.ones((n,)), np.ones((n,)), M)

    Tn = sum([S(X[i,], H[next((i for i, x in enumerate(G0[i,]) if x > 0), None),]) for i in range(n)])
    # For central, it follows |2*Binomial(n,1/2)-1|^2
    # For sign, it follows the independent sum of p |2*Binomial(n,1/2)-1|^2
    return(np.sum(Tn*Tn))

In [ ]:
# We apply the above functions to different simulated data below.
# The output is the empirical power, i.e., proportion of rejecting the null hypothesis.
# Examples C1-C10, S1-S10, Sp1-Sp10 are the same as those in Appendix D.2 in the supplementary file.

In [5]:
# Null distribution of OT-sign test
start_time = datetime.now()
def OneTn(i):
    np.random.seed(i)
    return(sum((np.random.binomial(200,0.5,size=50)*2 - 200)**2))
pool = mp.Pool(NUM_CORE)
Tns = np.array(pool.map(OneTn,range(10000)))
end_time = datetime.now()
print(end_time - start_time)

0:00:00.136267


In [ ]:
# Example S9: T2, OT-Wilcox, OT-sign tests (with random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
    H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.6508
0.2998
0:21:05.909296
0:05:38.685155
0.4949


In [7]:
# Example S9: T2, OT-Wilcox, OT-sign tests (with Halton reference vector)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    sampler = qmc.Halton(d=50,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])
    
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, H = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, H) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.6508
0.2809
0:20:40.474703
0:05:42.759016
0.4949


In [8]:
# Example C2: T2, OT-Wilcox tests (with Halton reference vectors)
from scipy.stats import multivariate_t

iter = 10000
if True:
    n = 200
    Result = np.zeros((5, 2))
    
    #pool = mp.Pool(NUM_CORE)
    # T2 CentralGaussian
    for j in range(5):
        loc = j/5
        sampler = qmc.Halton(d=2,scramble=False)
        U = sampler.random(n=201)[1:,:]
        H = np.zeros((200,2))
        H[:,1] = norm.ppf(U[:,1])
        H[:,0] = halfnorm.ppf(U[:,0])
        
        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter


        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t, H = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, multivariate_t, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.028 0.053]
[0.0279 0.1341]
[0.0446 0.475 ]
[0.0754 0.7934]
[0.1228 0.961 ]


In [9]:
# Example C3: T2, OT-Wilcox tests (with Halton reference vectors)
iter = 10000
from scipy.stats import uniform

if True:
    n = 200
    Result = np.zeros((5, 2))

    # T2 CentralGaussian
    for j in range(5):
        loc = j*0.03
        sampler = qmc.Halton(d=2,scramble=False)
        U = sampler.random(n=201)[1:,:]
        H = np.zeros((200,2))
        H[:,1] = norm.ppf(U[:,1])
        H[:,0] = halfnorm.ppf(U[:,0])
        
        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter


        def Reject(i_n_loc_u):
            i, n, loc, uniform, H = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, uniform, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0502 0.0391]
[0.0994 0.1803]
[0.2689 0.5746]
[0.547  0.8836]
[0.8113 0.9805]


In [10]:
# Example C1: OT-sign test (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 2))
   

    # T2 OT-sign 
    for j in range(5):
        loc = j/10
        sampler = qmc.Halton(d=2,scramble=False)
        U = sampler.random(n=201)[1:,:]
        H = np.zeros((200,2))
        H[:,1] = norm.ppf(U[:,1])
        H[:,0] = halfnorm.ppf(U[:,0])

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, H = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n) + loc
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0563 0.0566]
[0.1544 0.1135]
[0.4874 0.2862]
[0.8432 0.5427]
[0.9811 0.7748]


In [11]:
# Example C2: OT-sign test (Halton reference vectors)
from scipy.stats import multivariate_t

iter = 10000
if True:
    n = 200
    Result = np.zeros((5, 2))

    # T2, OT-sign test
    for j in range(5):
        loc = j/5
        sampler = qmc.Halton(d=2,scramble=False)
        U = sampler.random(n=201)[1:,:]
        H = np.zeros((200,2))
        H[:,1] = norm.ppf(U[:,1])
        H[:,0] = halfnorm.ppf(U[:,0])
        
        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter


        def Reject(i_n_loc_t):
            i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t, H = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t, H) for i in range(iter)])) / iter
        
        
        print(Result[j,])

[0.028  0.0504]
[0.0279 0.1845]
[0.0446 0.5391]
[0.0754 0.8659]
[0.1228 0.9748]


In [12]:
# Example C3: OT-sign test (Halton reference vectors)
iter = 10000
from scipy.stats import uniform

if True:
    n = 200
    Result = np.zeros((5, 2))

    # T2, OT-sign test
    for j in range(5):
        loc = j*0.03
        sampler = qmc.Halton(d=2,scramble=False)
        U = sampler.random(n=201)[1:,:]
        H = np.zeros((200,2))
        H[:,1] = norm.ppf(U[:,1])
        H[:,0] = halfnorm.ppf(U[:,0])        
        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter


        def Reject(i_n_loc_u):
            i, n, loc, Wilcoxon_sign, gram_schmidt, uniform, H = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, uniform, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0502 0.0544]
[0.0994 0.0897]
[0.2689 0.183 ]
[0.547  0.3397]
[0.8113 0.5196]


In [13]:
# Example Sp1: T2 and OT-Wilcox tests
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 2))
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]


    # T2, OT-Wilcox
    for j in range(5):
        loc = j*0.05

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + loc
            # spherical symmetry
            H = np.zeros((n, 2))
            H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
            W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter
        print(Result[j,])

[0.0563 0.0533]
[0.1451 0.1379]
[0.4234 0.4158]
[0.7708 0.7644]
[0.9522 0.9499]


In [14]:
# Example Sp2: T2 and OT-Wilcox tests
from scipy.stats import multivariate_t

if True:
    n = 200
    Result = np.zeros((5, 2))
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]
    
    # T2 SphericalGaussian BL
    for j in range(5):
        loc = j*0.1

        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter

        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t, H = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, multivariate_t, H) for i in range(iter)])) / iter
              
        print(Result[j,])

[0.028  0.0614]
[0.0503 0.1535]
[0.0559 0.4525]
[0.0671 0.7848]
[0.1034 0.9552]


In [15]:
# Example Sp3: T2 and OT-Wilcox tests
from scipy.stats import uniform

if True:
    n = 200
    Result = np.zeros((5, 2))
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]
    # T2 SphericalGaussian BL
    for j in range(5):
        loc = j*0.025

        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter

        def Reject(i_n_loc_u):
            i, n, loc, uniform, H = i_n_loc_u
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, uniform, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0559 0.0446]
[0.1369 0.1873]
[0.3967 0.5897]
[0.7516 0.8967]
[0.9553 0.9889]


In [16]:
# Example Sp1: OT-sign test (Halton reference vectors)
if True:
    n = 200
    Result = np.zeros((5, 2))
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]
    # T2 SphGaussian
    for j in range(5):
        loc = j*0.05

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, H = i_n_loc
            np.random.seed(i)
            X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + loc
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0563 0.0554]
[0.1451 0.0607]
[0.4234 0.1206]
[0.7708 0.215 ]
[0.9522 0.3657]


In [17]:
# Example Sp2: OT-sign test (Halton reference vectors)
from scipy.stats import multivariate_t
if True:
    n = 200
    Result = np.zeros((5, 2))
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]
    
    # T2 SphGaussian
    for j in range(5):
        loc = j*0.1

        def Reject(i_n_loc):
            i, n, loc, multivariate_t = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t, H = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.028  0.0564]
[0.0503 0.0886]
[0.0559 0.204 ]
[0.0671 0.3925]
[0.1034 0.6184]


In [18]:
# Example Sp3: OT-sign test (Halton reference vectors)
from scipy.stats import uniform
if True:
    n = 200
    Result = np.zeros((10, 2))
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]
   
    # T2 SphGaussian
    for j in range(5):
        loc = j*0.025

        def Reject(i_n_loc):
            i, n, loc, uniform = i_n_loc
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, uniform, H = i_n_loc
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, uniform, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0559 0.0463]
[0.1369 0.0603]
[0.3967 0.0876]
[0.7516 0.1455]
[0.9553 0.2313]


In [19]:
# Example S1: T2, OT-Wilcox, OT-sign tests (Halton reference vectors)
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    sampler = qmc.Halton(d=2,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])

    # T2 Wilcox Sign
    for j in range(5):
        loc = j*0.1
        
        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 0], [0, 3]], size=n) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter

        def Reject(i_n_loc):
            i, n, loc, H = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 0], [0, 3]], size=n) + loc
            W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)
        
        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, H) for i in range(iter)])) / iter
        
        def Reject(i_n_loc_u):
            i, n, loc, Wilcoxon_sign, gram_schmidt, uniform, H = i_n_loc_u
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 0], [0, 3]], size=n) + loc
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, uniform, H) for i in range(iter)])) / iter
        print(Result[j,])

[0.0563 0.0436 0.0458]
[0.2079 0.1782 0.1379]
[0.6387 0.5959 0.4326]
[0.9408 0.9257 0.7901]
[0.9976 0.9962 0.9656]


In [20]:
# Example S2: T2, OT-Wilcox, OT-sign tests (Halton reference vectors)
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    sampler = qmc.Halton(d=2,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])

    # T2 Wilcox Sign
    for j in range(5):
        loc = j*0.1
        
        def Reject(i_n_loc):
            i, n, loc, multivariate_t = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 0.0], [0.0, 3.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter

        def Reject(i_n_loc):
            i, n, loc, multivariate_t, H = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 0.0], [0.0, 3.0]], df=1, size=n)
            W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)
        
        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, multivariate_t, H) for i in range(iter)])) / iter
        
        def Reject(i_n_loc_u):
            i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t, H = i_n_loc_u
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 0.0], [0.0, 3.0]], df=1, size=n)
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t, H) for i in range(iter)])) / iter
        print(Result[j,])

[0.028  0.053  0.0419]
[0.0391 0.0949 0.117 ]
[0.0531 0.201  0.2932]
[0.0559 0.4328 0.5809]
[0.0615 0.6477 0.8325]


In [21]:
# Example S3: T2, OT-Wilcox, OT-sign tests (Halton reference vectors)
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    sampler = qmc.Halton(d=2,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])

    # T2 Wilcox Sign
    for j in range(5):
        loc = j*0.025

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.uniform(low=-1,high=1,size=(n,2)) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter

        def Reject(i_n_loc):
            i, n, loc, H = i_n_loc
            np.random.seed(i)
            X = np.random.uniform(low=-1,high=1,size=(n,2)) + loc
            W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)
        
        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, H) for i in range(iter)])) / iter
        
        def Reject(i_n_loc_u):
            i, n, loc, Wilcoxon_sign, gram_schmidt, H = i_n_loc_u
            np.random.seed(i)
            X = np.random.uniform(low=-1,high=1,size=(n,2)) + loc
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter
        print(Result[j,])

[0.0522 0.0416 0.0465]
[0.1172 0.1441 0.0682]
[0.3286 0.4648 0.1303]
[0.6455 0.7982 0.2502]
[0.8859 0.961  0.4174]


In [25]:
# Example Sp2: T2, OT-Wilcox tests (random reference vectors)
from scipy.stats import multivariate_t

iter = 10000
if True:
    n = 200
    Result = np.zeros((5, 2))
    
    # T2 SphericalGaussian
    for j in range(5):
        loc = j*0.1

        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter

        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            # spherical symmetry
            H = np.zeros((n, 2))
            H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
            W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter
              
        print(Result[j,])

[0.028  0.0655]
[0.0503 0.1575]
[0.0559 0.4615]
[0.0671 0.7875]
[0.1034 0.9562]


In [26]:
# Example Sp1: T2, OT-Wilcox tests (random reference vectors)
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 2))
   

    # T2 SphericalGaussian
    for j in range(5):
        loc = j*0.05
        
        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.normal(loc=0.0, scale=1.0, size=(n, 2)) + loc
            # spherical symmetry
            H = np.zeros((n, 2))
            H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
            W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter
        print(Result[j,])

[0.0563 0.0533]
[0.1451 0.1379]
[0.4234 0.4158]
[0.7708 0.7644]
[0.9522 0.9499]


In [27]:
# Example Sp3: T2, OT-Wilcox tests (random reference vectors)
iter = 10000
from scipy.stats import uniform

if True:
    n = 200
    Result = np.zeros((5, 2))

    # T2 SphericalGaussian
    for j in range(5):
        loc = j*0.025

        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter

        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            # spherical symmetry
            H = np.zeros((n, 2))
            H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
            W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0559 0.0547]
[0.1369 0.2056]
[0.3967 0.6145]
[0.7516 0.9029]
[0.9553 0.9911]


In [29]:
# Example C1: T2, OT-Wilcox tests (random reference vectors)
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 2))
   

    # T2 CentralGaussian 
    for j in range(5):
        loc = j/10

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n) + loc
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, 2))
            H[:, 0] = abs(H[:, 0])
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter
        
        print(Result[j,])
        #np.savetxt('/Users/huangzhen/desktop/HPC/Python/CenGaussian.csv', Result, delimiter=',')

[0.0563 0.0539]
[0.1544 0.146 ]
[0.4874 0.4591]
[0.8432 0.8164]
[0.9811 0.975 ]


In [79]:
# Example C1: T2, OT-Wilcox tests (Halton reference vectors)
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 2))
    sampler = qmc.Halton(d=2,scramble=False)
    U = sampler.random(n=201)[1:,:]
    H = np.zeros((200,2))
    H[:,1] = norm.ppf(U[:,1])
    H[:,0] = halfnorm.ppf(U[:,0])

    # T2 CentralGaussian 
    for j in range(5):
        loc = j/10

        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n) + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, H = i_n_loc
            np.random.seed(i)
            X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n) + loc
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, H) for i in range(iter)])) / iter
        
        print(Result[j,])
        #np.savetxt('/Users/huangzhen/desktop/HPC/Python/CenGaussian.csv', Result, delimiter=',')

[0.0563 0.0434]
[0.1544 0.1286]
[0.4874 0.4249]
[0.8432 0.7989]
[0.9811 0.9704]


In [30]:
# Example C2: T2, OT-Wilcox tests (random reference vectors)
from scipy.stats import multivariate_t

iter = 10000
if True:
    n = 200
    Result = np.zeros((5, 2))
    #pool = mp.Pool(NUM_CORE)
    # T2 CentralGaussian
    for j in range(5):
        loc = j/5

        
        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter


        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, 2))
            H[:, 0] = abs(H[:, 0])
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.028  0.0617]
[0.0279 0.1535]
[0.0446 0.4768]
[0.0754 0.8053]
[0.1228 0.9618]


In [31]:
# Example C3: T2, OT-Wilcox tests (random reference vectors)
iter = 10000
from scipy.stats import uniform

if True:
    n = 200
    Result = np.zeros((5, 2))

    # T2 CentralGaussian
    for j in range(5):
        loc = j*0.03
        
        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter


        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, 2))
            H[:, 0] = abs(H[:, 0])
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0502 0.048 ]
[0.0994 0.2201]
[0.2689 0.6374]
[0.547  0.9115]
[0.8113 0.9858]


In [32]:
# Example C2: T2, OT-sign tests (random reference vectors)
from scipy.stats import multivariate_t

iter = 10000
if True:
    n = 200
    Result = np.zeros((5, 2))

    # T2 CentralGaussian
    for j in range(5):
        loc = j/5

        
        def Reject(i_n_loc_t):
            i, n, loc, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter


        def Reject(i_n_loc_t):
            i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t = i_n_loc_t
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 1.0], [1.0, 3.0]], df=1, size=n)
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, 2))
            H[:, 0] = abs(H[:, 0])
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t) for i in range(iter)])) / iter
        
        
        print(Result[j,])

[0.028  0.0507]
[0.0279 0.1856]
[0.0446 0.5448]
[0.0754 0.8597]
[0.1228 0.975 ]


In [33]:
# Example C3: T2, OT-sign tests (random reference vectors)
iter = 10000
from scipy.stats import uniform

if True:
    n = 200
    Result = np.zeros((5, 2))

    # T2 CentralGaussian
    for j in range(5):
        loc = j*0.03
        
        def Reject(i_n_loc_u):
            i, n, loc, uniform = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter


        def Reject(i_n_loc_u):
            i, n, loc, Wilcoxon_sign, gram_schmidt, uniform = i_n_loc_u
            np.random.seed(i)
            X = uniform.rvs(scale=1,size=(n,2))
            s = np.random.binomial(1, 0.5, n)*2 - 1
            X[:,0] = X[:,0] * s
            X[:,1] = X[:,1] * s
            X = X + loc
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, 2))
            H[:, 0] = abs(H[:, 0])
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, uniform) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0502 0.0533]
[0.0994 0.0894]
[0.2689 0.1859]
[0.547  0.3483]
[0.8113 0.536 ]


In [34]:
# Example Sp2: T2, OT-sign tests (random reference vectors)
from scipy.stats import multivariate_t
iter = 10000
if True:
    n = 200
    Result = np.zeros((5, 2))
   
    # T2 SphGaussian
    for j in range(5):
        loc = j*0.1

        def Reject(i_n_loc):
            i, n, loc, multivariate_t = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[1.0, 0.0], [0.0, 1.0]], df=1, size=n)
            # spherical symmetry
            H = np.zeros((n, 2))
            H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.028  0.0536]
[0.0503 0.0893]
[0.0559 0.2045]
[0.0671 0.3945]
[0.1034 0.6216]


In [35]:
# Example Sp3: T2, OT-sign tests (random reference vectors)
from scipy.stats import uniform
iter = 10000
if True:
    n = 200
    Result = np.zeros((5, 2))
   
    # T2 SphGaussian
    for j in range(5):
        loc = j*0.025

        def Reject(i_n_loc):
            i, n, loc, uniform = i_n_loc
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, uniform) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, uniform = i_n_loc
            np.random.seed(i)
            X = np.zeros((n, 2))
            r = np.sqrt(uniform.rvs(scale=1,size=n))
            theta = uniform.rvs(scale =2*np.pi,size=n)
            X[:,0] = r*np.cos(theta)
            X[:,1] = r*np.sin(theta)
            X = X + loc
            # spherical symmetry
            H = np.zeros((n, 2))
            H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
                return (1)
            else:
                return (0)


        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, uniform) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0559 0.0549]
[0.1369 0.0605]
[0.3967 0.0899]
[0.7516 0.1386]
[0.9553 0.2281]


In [36]:
# Example S2: T2, OT-Wilcox, OT-sign tests (random reference vectors)
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
   

    # T2 Wilcox Sign
    for j in range(5):
        loc = j*0.1
        
        def Reject(i_n_loc):
            i, n, loc, multivariate_t = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 0.0], [0.0, 3.0]], df=1, size=n)
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter

        def Reject(i_n_loc):
            i, n, loc, multivariate_t = i_n_loc
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 0.0], [0.0, 3.0]], df=1, size=n)
            H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, 2)))
            W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
            if chi2.cdf(np.inner(W, W), df=2) > 0.95:
                return (1)
            else:
                return (0)
        
        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, multivariate_t) for i in range(iter)])) / iter
        
        def Reject(i_n_loc_u):
            i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t = i_n_loc_u
            np.random.seed(i)
            X = multivariate_t.rvs(loc=[loc, loc], shape=[[2.0, 0.0], [0.0, 3.0]], df=1, size=n)
            H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, 2)))
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, multivariate_t) for i in range(iter)])) / iter
        print(Result[j,])

[0.028  0.0605 0.0419]
[0.0391 0.1034 0.117 ]
[0.0531 0.2222 0.2932]
[0.0559 0.4377 0.5809]
[0.0615 0.6586 0.8325]


In [40]:
# Examples C4 and C5: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000

if True:
    #pool = mp.Pool(NUM_CORE)
    Result = np.zeros((5, 3))
   

    # T2 Sign Signed rank
    for j in range(2):
        loc = 0
        n = 100*(j+1)

        def Reject(i_n_loc):
            i, n, loc, f = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.exponential(scale=1.0, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.exponential(scale=1.0, size=n) - 1
            X = X + loc
            
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, f) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.exponential(scale=1.0, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.exponential(scale=1.0, size=n) - 1
            X = X + loc
                
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
            H[:, 0] = abs(H[:, 0])
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt) for i in range(iter)])) / iter
        
        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.exponential(scale=1.0, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.exponential(scale=1.0, size=n) - 1
            X = X + loc
                
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
            H[:, 0] = abs(H[:, 0])
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=d) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0677 0.6319 0.1003]
[0.0613 0.9058 0.1505]


In [41]:
# Examples C4 and C5: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000

if True:
    #pool = mp.Pool(NUM_CORE)
    Result = np.zeros((5, 3))

    # T2 Sign Signed rank
    for j in range(2):
        loc = 0
        n = 100*(j+1)
        sampler = qmc.Halton(d=2,scramble=False)
        U = sampler.random(n=n+1)[1:,:]
        H = np.zeros((n,2))
        H[:,1] = norm.ppf(U[:,1])
        H[:,0] = halfnorm.ppf(U[:,0])

        def Reject(i_n_loc):
            i, n, loc, f = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.exponential(scale=1.0, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.exponential(scale=1.0, size=n) - 1
            X = X + loc
            
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, f) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, H = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.exponential(scale=1.0, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.exponential(scale=1.0, size=n) - 1
            X = X + loc
                
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter
        
        def Reject(i_n_loc):
            i, n, loc, H = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.exponential(scale=1.0, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.exponential(scale=1.0, size=n) - 1
            X = X + loc

            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=d) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.0677 0.6602 0.088 ]
[0.0613 0.9271 0.1519]


In [42]:
# Example C7: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000

if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
   

    # T2 Sign Signed rank
    for j in range(1):
        loc = j/10

        def Reject(i_n_loc):
            i, n, loc, f = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.pareto(a=1.0, size=n) - 2
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.pareto(a=1.0, size=n) - 2
            X = X + loc
            
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, f) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.pareto(a=1.0, size=n) - 2
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.pareto(a=1.0, size=n) - 2
            X = X + loc
                
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
            H[:, 0] = abs(H[:, 0])
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt) for i in range(iter)])) / iter
        
        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.pareto(a=1.0, size=n) - 2
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.pareto(a=1.0, size=n) - 2
            X = X + loc
                
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
            H[:, 0] = abs(H[:, 0])
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=d) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.837  0.8972 0.0445]


In [43]:
# Example C7: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000

if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    sampler = qmc.Halton(d=2,scramble=False)
    U = sampler.random(n=201)[1:,:]
    H = np.zeros((200,2))
    H[:,1] = norm.ppf(U[:,1])
    H[:,0] = halfnorm.ppf(U[:,0])

    # T2 Sign Signed rank
    for j in range(1):
        loc = j/10

        def Reject(i_n_loc):
            i, n, loc, f = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.pareto(a=1.0, size=n) - 2
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.pareto(a=1.0, size=n) - 2
            X = X + loc
            
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, f) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, H = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.pareto(a=1.0, size=n) - 2
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.pareto(a=1.0, size=n) - 2
            X = X + loc
            
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter
        
        def Reject(i_n_loc):
            i, n, loc, H = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.pareto(a=1.0, size=n) - 2
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.pareto(a=1.0, size=n) - 2
            X = X + loc
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=d) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.837  0.9566 0.0218]


In [44]:
# Example C6: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000

if True:
    #pool = mp.Pool(NUM_CORE)
    n = 100
    Result = np.zeros((5, 3))
   

    # T2 Sign Signed rank
    for j in range(1):
        loc = j/10

        def Reject(i_n_loc):
            i, n, loc, f = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.chisquare(df=1, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.chisquare(df=1, size=n) - 1
            X = X + loc
            
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, f) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.chisquare(df=1, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.chisquare(df=1, size=n) - 1
            X = X + loc
                
            # central symmetry
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
            H[:, 0] = abs(H[:, 0])
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt) for i in range(iter)])) / iter
        
        def Reject(i_n_loc):
            i, n, loc = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.chisquare(df=1, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.chisquare(df=1, size=n) - 1
            X = X + loc
                
            H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
            H[:, 0] = abs(H[:, 0])
            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=d) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc) for i in range(iter)])) / iter
        
        print(Result[j,])
# n=100
# [0.086 0.842 0.178]

[0.085  0.8288 0.2064]


In [45]:
# Example C6: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000

if True:
    #pool = mp.Pool(NUM_CORE)
    n = 100
    Result = np.zeros((5, 3))
    sampler = qmc.Halton(d=2,scramble=False)
    U = sampler.random(n=101)[1:,:]
    H = np.zeros((100,2))
    H[:,1] = norm.ppf(U[:,1])
    H[:,0] = halfnorm.ppf(U[:,0])

    # T2 Sign Signed rank
    for j in range(1):
        loc = j/10

        def Reject(i_n_loc):
            i, n, loc, f = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.chisquare(df=1, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.chisquare(df=1, size=n) - 1
            X = X + loc
            
            # Hotelling's t2
            mu = X.mean(axis=0)
            Sigma = np.cov(X, rowvar=False)
            if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=2) > 0.95:
                return (1)
            else:
                return (0)

        Result[j, 0] = sum(pool.map(Reject, [(i, n, loc, f) for i in range(iter)])) / iter


        def Reject(i_n_loc):
            i, n, loc, Wilcoxon_sign, gram_schmidt, H = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.chisquare(df=1, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.chisquare(df=1, size=n) - 1
            X = X + loc
            if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Central") <= 0.05:
                return (1)
            else:
                return (0)

        Result[j, 1] = sum(pool.map(Reject, [(i, n, loc, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter
        
        def Reject(i_n_loc):
            i, n, loc, H = i_n_loc
            np.random.seed(i)
            d = 2
            X = np.zeros([n,d])
            phi = 0
            X[:,0] = np.random.chisquare(df=1, size=n) - 1
            for j in range(1,d):
                X[:,j] = X[:,j-1]*phi + np.random.chisquare(df=1, size=n) - 1
            X = X + loc

            W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
            if chi2.cdf(np.inner(W, W), df=d) > 0.95:
                return (1)
            else:
                return (0)


        Result[j, 2] = sum(pool.map(Reject, [(i, n, loc, H) for i in range(iter)])) / iter
        
        print(Result[j,])

[0.085  0.8754 0.1932]


In [46]:
# Example C10: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
from scipy.stats import binom
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    
    d = 50
    H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
    
    
    phi = 0.5
    # T2
    def Reject(i_n_loc):
        i, n, d, f, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.standard_t(df=1, size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.standard_t(df=1, size=n)
        X = X + 0.9
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, phi) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.standard_t(df=1, size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.standard_t(df=1, size=n)
        X = X + 0.9       
        # central symmetry
        H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
        H[:, 0] = abs(H[:, 0])
        Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Central")
        # For central, it follows |2*Binomial(n,1/2)-n|^2
        CI = (2*np.array(binom.interval(0.95, n, 1/2))-n)**2
        if Tn > CI[1]:
            return(1)
        else:
            return(0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.standard_t(df=1, size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.standard_t(df=1, size=n)
        X = X + 0.9        
        H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
        H[:, 0] = abs(H[:, 0])
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H * ((np.random.binomial(1, 0.5, n)*2 - 1)[:,np.newaxis]))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, phi) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.3491
0.0637
0.2382
0:11:21.283643


In [47]:
# Example C10: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
from scipy.stats import binom
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    d = 50
    sampler = qmc.Halton(d=50,scramble=False)
    U = sampler.random(n=201)[1:,:]
    H = np.zeros((200,50))
    H[:,1:] = norm.ppf(U[:,1:])
    H[:,0] = halfnorm.ppf(U[:,0])
    
    
    phi = 0.5
    # T2
    def Reject(i_n_loc):
        i, n, d, f, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.standard_t(df=1, size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.standard_t(df=1, size=n)
        X = X + 0.9
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, phi) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom, H = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.standard_t(df=1, size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.standard_t(df=1, size=n)
        X = X + 0.9       
        Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Central")
        # For central, it follows |2*Binomial(n,1/2)-n|^2
        CI = (2*np.array(binom.interval(0.95, n, 1/2))-n)**2
        if Tn > CI[1]:
            return(1)
        else:
            return(0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom, H) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, phi, H = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.standard_t(df=1, size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.standard_t(df=1, size=n)
        X = X + 0.9        
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H * ((np.random.binomial(1, 0.5, n)*2 - 1)[:,np.newaxis]))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, phi, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.3491
0.2291
0.8857
0:11:44.261211


In [48]:
# Example Sp4: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    
    # T2
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[4, 0], [0, 1]], size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[4, 0], [0, 1]], size=n)
        H = np.zeros((n, 2))
        H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
            return (1)
        else:
            return (0)
        
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[4, 0], [0, 1]], size=n)
        H = np.zeros((n, 2))
        H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0563
0.0517
0.063
0:00:08.563189


In [49]:
# Example Sp4: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]
    # T2
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[4, 0], [0, 1]], size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[4, 0], [0, 1]], size=n)
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
            return (1)
        else:
            return (0)
        
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[4, 0], [0, 1]], size=n)
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0563
0.0547
0.0584
0:00:10.638188


In [50]:
# Example Sp5: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    
    # T2
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0.6], [0.6, 1]], size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0.6], [0.6, 1]], size=n)
        H = np.zeros((n, 2))
        H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
            return (1)
        else:
            return (0)
        
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0.6], [0.6, 1]], size=n)
        H = np.zeros((n, 2))
        H[:, 0] = np.sqrt(np.random.chisquare(df=2, size=n))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0563
0.0501
0.063
0:00:09.783399


In [51]:
# Example Sp5: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=2))[:,0]
    # T2
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0.6], [0.6, 1]], size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0.6], [0.6, 1]], size=n)
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
            return (1)
        else:
            return (0)
        
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0.6], [0.6, 1]], size=n)
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0563
0.0532
0.0584
0:00:09.951098


In [52]:
# Example Sp6: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 1000
    d = 2
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((1000,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=1001)[1:,:],df=2))[:,0]
    # T2
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.uniform(low=-1,high=1,size=(n,2))
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.uniform(low=-1,high=1,size=(n,2))
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
            return (1)
        else:
            return (0)
        
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.uniform(low=-1,high=1,size=(n,2))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0469
0.0477
0.0471
0:00:59.753248


In [53]:
# Example Sp7: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 100
    d = 2
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((100,2))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=101)[1:,:],df=2))[:,0]
    # T2
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.chisquare(df=1, size=(n,2)) - 1
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.chisquare(df=1, size=(n,2)) - 1
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Spherical") <= 0.05:
            return (1)
        else:
            return (0)
        
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.chisquare(df=1, size=(n,2)) - 1
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0871
0.4528
0.2536
0:00:05.233907


In [54]:
# Null distribution for OT-sign test (spherical symmetry)
from scipy.stats import ortho_group
from datetime import datetime
start_time = datetime.now()
def OneTn(i):
    np.random.seed(i)
    Sums = sum(ortho_group.rvs(dim=50,size=200))
    return(np.sum(Sums*Sums))
Tns = np.array(pool.map(OneTn,range(10000)))
end_time = datetime.now()
print(end_time - start_time)

0:02:00.245994


In [55]:
# Example Sp8: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
from scipy.stats import ortho_group
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        H = np.zeros((n, d))
        H[:, 0] = np.sqrt(np.random.chisquare(df=d, size=n))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        def my_func(x):
            return(x/np.sqrt(np.sum(x*x)))
        for j in range(1000):
            Unit_vec = np.apply_along_axis(my_func, 1, np.random.normal(size=(n,d)))
            Wj = sum((H[:,0][:,np.newaxis])*Unit_vec)
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

# Sign test
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group = i_n_loc
    np.random.seed(i)
    X = np.random.normal(loc=0,scale=1,size=(n,d))
    H = np.zeros((n, d))
    H[:, 0] = np.sqrt(np.random.chisquare(df=d, size=n))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Spherical")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0435
0.0518
1:54:06.089423
1:45:18.250747
0.0462


In [56]:
# Example Sp8: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
from scipy.stats import ortho_group
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,50))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=d))[:,0]
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        def my_func(x):
            return(x/np.sqrt(np.sum(x*x)))
        for j in range(1000):
            Unit_vec = np.apply_along_axis(my_func, 1, np.random.normal(size=(n,d)))
            Wj = sum((H[:,0][:,np.newaxis])*Unit_vec)
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

# Sign test
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group, H = i_n_loc
    np.random.seed(i)
    X = np.random.normal(loc=0,scale=1,size=(n,d))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Spherical")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group, H) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0435
0.0511
1:51:15.568799
1:44:56.809225
0.0455


In [57]:
# Example Sp9: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        X = X + 0.05
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        X = X + 0.05
        H = np.zeros((n, d))
        H[:, 0] = np.sqrt(np.random.chisquare(df=d, size=n))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        def my_func(x):
            return(x/np.sqrt(np.sum(x*x)))
        for j in range(1000):
            Unit_vec = np.apply_along_axis(my_func, 1, np.random.normal(size=(n,d)))
            Wj = sum((H[:,0][:,np.newaxis])*Unit_vec)
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group = i_n_loc
    np.random.seed(i)
    X = np.random.normal(loc=0,scale=1,size=(n,d))
    X = X + 0.05
    H = np.zeros((n, d))
    H[:, 0] = np.sqrt(np.random.chisquare(df=d, size=n))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Spherical")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.5425
0.6827
1:51:42.965095
1:45:05.442092
0.046


In [58]:
# Example Sp9: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,50))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=d))[:,0]
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        X = X + 0.05
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.normal(loc=0,scale=1,size=(n,d))
        X = X + 0.05
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        def my_func(x):
            return(x/np.sqrt(np.sum(x*x)))
        for j in range(1000):
            Unit_vec = np.apply_along_axis(my_func, 1, np.random.normal(size=(n,d)))
            Wj = sum((H[:,0][:,np.newaxis])*Unit_vec)
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group, H = i_n_loc
    np.random.seed(i)
    X = np.random.normal(loc=0,scale=1,size=(n,d))
    X = X + 0.05
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Spherical")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group, H) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.5425
0.6825
1:51:21.641504
1:45:57.061015
0.051


In [59]:
# Example Sp10: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
            
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    # T2
    def Reject(i_n_loc):
        i, n, d, f, multivariate_t = i_n_loc
        np.random.seed(i)
        X = multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n)
        X = X + 0.05
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, multivariate_t) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, multivariate_t = i_n_loc
        np.random.seed(i)
        X = multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n)
        X = X + 0.05
        H = np.zeros((n, d))
        H[:, 0] = np.sqrt(np.random.chisquare(df=d, size=n))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        def my_func(x):
            return(x/np.sqrt(np.sum(x*x)))
        for j in range(1000):
            Unit_vec = np.apply_along_axis(my_func, 1, np.random.normal(size=(n,d)))
            Wj = sum((H[:,0][:,np.newaxis])*Unit_vec)
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, multivariate_t) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group, multivariate_t = i_n_loc
    np.random.seed(i)
    X = multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n)
    X = X + 0.05
    H = np.zeros((n, d))
    H[:, 0] = np.sqrt(np.random.chisquare(df=d, size=n))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Spherical")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, ortho_group, multivariate_t) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0531
0.3471
1:53:35.187005
1:47:34.482104
0.0487


In [60]:
# Example Sp10: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
            
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    sampler = qmc.Halton(d=1,scramble=False)
    H = np.zeros((200,50))
    H[:,0] = np.sqrt(chi2.ppf(sampler.random(n=201)[1:,:],df=d))[:,0]
    # T2
    def Reject(i_n_loc):
        i, n, d, f, multivariate_t = i_n_loc
        np.random.seed(i)
        X = multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n)
        X = X + 0.05
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, multivariate_t) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, multivariate_t, H = i_n_loc
        np.random.seed(i)
        X = multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n)
        X = X + 0.05
        W = Wilcoxon_signed_rank(X, H, Symmetry="Spherical")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        def my_func(x):
            return(x/np.sqrt(np.sum(x*x)))
        for j in range(1000):
            Unit_vec = np.apply_along_axis(my_func, 1, np.random.normal(size=(n,d)))
            Wj = sum((H[:,0][:,np.newaxis])*Unit_vec)
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, multivariate_t, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, multivariate_t, H = i_n_loc
    np.random.seed(i)
    X = multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n)
    X = X + 0.05
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Spherical")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, multivariate_t, H) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0531
0.3488
1:55:29.849609
1:53:14.717787
0.0521


In [61]:
# Example S4: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n)
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, 2)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n)
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, 2)))
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0563
0.058
0.0519
0:11:39.828090


In [62]:
# Example S4: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    sampler = qmc.Halton(d=2,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n)
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.multivariate_normal(mean=[0,0], cov=[[2, 1], [1, 3]], size=n)
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0563
0.0488
0.0519
0:11:24.314109


In [63]:
# Example S7: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.laplace(loc=0.2, scale=1.0, size=(n,2))
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.random.laplace(loc=0.2, scale=1.0, size=(n,2))
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, 2)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
    # Sign
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt = i_n_loc
        np.random.seed(i)
        X = np.random.laplace(loc=0.2, scale=1.0, size=(n,2))
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, 2)))
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.7316
0.8112
0:05:47.505561
0.9156
0:05:27.682939


In [64]:
# Example S7: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 2
    sampler = qmc.Halton(d=2,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.laplace(loc=0.2, scale=1.0, size=(n,2))
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.laplace(loc=0.2, scale=1.0, size=(n,2))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
    # Sign
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.laplace(loc=0.2, scale=1.0, size=(n,2))
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.7316
0.7949
0:05:46.787270
0.9156
0:05:30.128149


In [65]:
# Example S5: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 100
    d = 2
    sampler = qmc.Halton(d=2,scramble=False)
    H = halfnorm.ppf(sampler.random(n=101)[1:,:])
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.exponential(scale=1.0, size=(n,2)) - 1
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.exponential(scale=1.0, size=(n,2)) - 1
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
    # Sign
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.exponential(scale=1.0, size=(n,2)) - 1
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.069
0.1239
0:01:28.349411
0.9309
0:01:23.407806


In [66]:
# Example S6: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 100
    d = 2
    sampler = qmc.Halton(d=2,scramble=False)
    H = halfnorm.ppf(sampler.random(n=101)[1:,:])
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.random.chisquare(df=1, size=(n,2)) - 1
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if chi2.cdf(n * np.inner(mu, np.linalg.solve(Sigma, mu)), df=d) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.random.chisquare(df=1, size=(n,2)) - 1
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        if chi2.cdf(np.inner(W, W), df=2) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
    # Sign
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign, gram_schmidt, H = i_n_loc
        np.random.seed(i)
        X = np.random.chisquare(df=1, size=(n,2)) - 1
        if Wilcoxon_sign(X, H, gram_schmidt, Symmetry="Sign") <= 0.05:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign, gram_schmidt, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0871
0.2788
0:01:27.391178
0.9984
0:01:24.034512


In [67]:
# Null distribution of OT-sign test (sign symmetry)
start_time = datetime.now()
def OneTn(i):
    np.random.seed(i)
    return(sum((np.random.binomial(200,0.5,size=50)*2 - 200)**2))
Tns = np.array(pool.map(OneTn,range(10000)))
end_time = datetime.now()
print(end_time - start_time)

0:00:00.113725


In [68]:
# Example S8: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d))
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d))
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d))
    H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0435
0.05
0:20:25.978683
0:05:37.165089
0.0482


In [69]:
# Example S8: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    sampler = qmc.Halton(d=50,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d))
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, H = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, H) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0435
0.0525
0:20:28.825478
0:06:22.536584
0.0482


In [70]:
# Example S9: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
    H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.6508
0.2998
0:21:27.871117
0:06:21.032489
0.4949


In [71]:
# Example S9: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    sampler = qmc.Halton(d=50,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])
    # T2
    def Reject(i_n_loc):
        i, n, d, f = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, H = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, H = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * np.random.normal(loc=0,scale=1,size=(n,d)) + 0.003
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, H) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.6508
0.2809
0:22:35.986409
0:06:06.710446
0.4949


In [72]:
# Example S10: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    # T2
    def Reject(i_n_loc):
        i, n, d, f, multivariate_t = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n) + 0.003
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, multivariate_t) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, multivariate_t = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n) + 0.003
        H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, multivariate_t) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, multivariate_t = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n) + 0.003
    H = abs(np.random.normal(loc=0.0, scale=1.0, size=(n, d)))
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, multivariate_t) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0476
0.1673
0:22:27.508477
0:06:44.496320
0.2823


In [73]:
# Example S10: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    d = 50
    sampler = qmc.Halton(d=50,scramble=False)
    H = halfnorm.ppf(sampler.random(n=201)[1:,:])
    # T2
    def Reject(i_n_loc):
        i, n, d, f, multivariate_t = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n) + 0.003
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, multivariate_t) for i in range(iter)])) / iter)
    
    # Signed rank
    start_time = datetime.now()
    def Reject(i_n_loc):
        i, n, d, multivariate_t, H = i_n_loc
        np.random.seed(i)
        X = np.sin(np.arange(1,d+1)) * multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n) + 0.003
        W = Wilcoxon_signed_rank(X, H, Symmetry="Sign")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H*(np.random.binomial(1,0.5,size=(n,d))*2 - 1))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, multivariate_t, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)
    
start_time = datetime.now()
def Reject(i_n_loc):
    i, n, d, Wilcoxon_sign_stat, gram_schmidt, multivariate_t, H = i_n_loc
    np.random.seed(i)
    X = np.sin(np.arange(1,d+1)) * multivariate_t.rvs(loc=[0.0]*d, shape=np.diag([1.0]*d), df=1, size=n) + 0.003
    Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Sign")
    return(Tn)
Observed_Tn = np.array(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, multivariate_t, H) for i in range(iter)]))
end_time = datetime.now()
print(end_time - start_time)
s = 0
for i in range(iter):
    if np.mean(Observed_Tn[i] > Tns) > 0.95:
        s = s + 1
print(s/iter)

0.0476
0.1264
0:21:46.028576
0:06:34.561294
0.2823


In [74]:
# Example C9: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
from scipy.stats import binom
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    
    d = 50 
    phi = 0.5
    # T2
    def Reject(i_n_loc):
        i, n, d, f, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        X = X + 0.15
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, phi) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        X = X + 0.15       
        # central symmetry
        H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
        H[:, 0] = abs(H[:, 0])
        Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Central")
        # For central, it follows |2*Binomial(n,1/2)-n|^2
        CI = (2*np.array(binom.interval(0.95, n, 1/2))-n)**2
        if Tn > CI[1]:
            return(1)
        else:
            return(0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        X = X + 0.15        
        H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
        H[:, 0] = abs(H[:, 0])
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H * ((np.random.binomial(1, 0.5, n)*2 - 1)[:,np.newaxis]))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, phi) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.972
0.0682
0.4146
0:11:57.366565


In [76]:
# Example C9: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
from scipy.stats import binom
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    
    d = 50 
    phi = 0.5
    sampler = qmc.Halton(d=50,scramble=False)
    U = sampler.random(n=201)[1:,:]
    H = np.zeros((200,50))
    H[:,1:] = norm.ppf(U[:,1:])
    H[:,0] = halfnorm.ppf(U[:,0])
    # T2
    def Reject(i_n_loc):
        i, n, d, f, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        X = X + 0.15
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, phi) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom, H = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        X = X + 0.15       
        Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Central")
        # For central, it follows |2*Binomial(n,1/2)-n|^2
        CI = (2*np.array(binom.interval(0.95, n, 1/2))-n)**2
        if Tn > CI[1]:
            return(1)
        else:
            return(0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom, H) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, phi, H = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        X = X + 0.15        
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H * ((np.random.binomial(1, 0.5, n)*2 - 1)[:,np.newaxis]))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, phi, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.972
0.1642
0.9939
0:11:12.856782


In [77]:
# Example C8: T2, OT-sign, OT-Wilcox tests (random reference vectors)
iter = 10000
from scipy.stats import binom
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    
    d = 50 
    phi = 0.5
    # T2
    def Reject(i_n_loc):
        i, n, d, f, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, phi) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)     
        # central symmetry
        H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
        H[:, 0] = abs(H[:, 0])
        Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Central")
        # For central, it follows |2*Binomial(n,1/2)-n|^2
        CI = (2*np.array(binom.interval(0.95, n, 1/2))-n)**2
        if Tn > CI[1]:
            return(1)
        else:
            return(0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)    
        H = np.random.normal(loc=0.0, scale=1.0, size=(n, d))
        H[:, 0] = abs(H[:, 0])
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H * ((np.random.binomial(1, 0.5, n)*2 - 1)[:,np.newaxis]))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, phi) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0465
0.0388
0.0501
0:10:53.772707


In [78]:
# Example C8: T2, OT-sign, OT-Wilcox tests (Halton reference vectors)
iter = 10000
from scipy.stats import binom
from datetime import datetime
if True:
    #pool = mp.Pool(NUM_CORE)
    n = 200
    Result = np.zeros((5, 3))
    
    d = 50 
    phi = 0.5
    sampler = qmc.Halton(d=50,scramble=False)
    U = sampler.random(n=201)[1:,:]
    H = np.zeros((200,50))
    H[:,1:] = norm.ppf(U[:,1:])
    H[:,0] = halfnorm.ppf(U[:,0])
    # T2
    def Reject(i_n_loc):
        i, n, d, f, phi = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        # Hotelling's t2
        mu = X.mean(axis=0)
        Sigma = np.cov(X, rowvar=False)
        if f.cdf(d,n-d,n * np.inner(mu, np.linalg.solve(Sigma, mu)) * (n-d)/d/(n-1)) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, f, phi) for i in range(iter)])) / iter)
    # Sign
    def Reject(i_n_loc):
        i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom, H = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)
        Tn = Wilcoxon_sign_stat(X, H, gram_schmidt, Symmetry="Central")
        # For central, it follows |2*Binomial(n,1/2)-n|^2
        CI = (2*np.array(binom.interval(0.95, n, 1/2))-n)**2
        if Tn > CI[1]:
            return(1)
        else:
            return(0)
    print(sum(pool.map(Reject, [(i, n, d, Wilcoxon_sign_stat, gram_schmidt, phi, binom, H) for i in range(iter)])) / iter)
    
    start_time = datetime.now()
    # Signed rank
    def Reject(i_n_loc):
        i, n, d, phi, H = i_n_loc
        np.random.seed(i)
        X = np.zeros([n,d])
        X[:,0] = np.random.normal(loc=0,scale=1,size=n)
        for j in range(1,d):
            X[:,j] = X[:,j-1]*phi + np.random.normal(loc=0,scale=1,size=n)    
        W = Wilcoxon_signed_rank(X, H, Symmetry="Central")
        Tn = np.inner(W, W)
        # Null distribution of Tn
        Tns = np.zeros(1000)
        for j in range(1000):
            Wj = sum(H * ((np.random.binomial(1, 0.5, n)*2 - 1)[:,np.newaxis]))
            Tns[j] = np.inner(Wj, Wj)
        Tns = Tns/n
        
        if np.mean(Tn > Tns) > 0.95:
            return (1)
        else:
            return (0)
    print(sum(pool.map(Reject, [(i, n, d, phi, H) for i in range(iter)])) / iter)
    end_time = datetime.now()
    print(end_time - start_time)

0.0465
0.041
0.0489
0:10:56.856432
